This code is developed based on the ArcGIS Pro spatial analysing procedure provide by Dr. Sherman Horn and Dr. Justin Tran.
The origional procedure is time consuming in ArcGIS Pro, the code assambles the all the methods in the procedure and can
finish the analysis within a short period of time.

The result of this code will be enter to a excel spreadsheet to calculate the total maize land.

The testing data for this code is provided by Dr. Amy Thompson at the Maya site named Xnaheb

In [5]:
#Import the arcpy package and set the workspace

In [1]:
import arcpy
from arcpy import env
from arcpy.sa import *

In [2]:
path = r'C:\Users\1017d\Box\GIS_class_folders\Final'

In [3]:
env.workspace = path

In [4]:
env.overwriteOutput = True

In [6]:
#Check the raster file and the shapefile in the folder
#The original folder should only contain a DEM.tif raster, a housegroup.shp shapefile and a outline.shp shapeflie.

In [7]:
arcpy.ListRasters()

['30mSTRMdem2.tif', '30mSTRMdemHillshade.tif', 'DEM.tif']

In [8]:
arcpy.ListFeatureClasses()

['15polygon.shp',
 '15_land.shp',
 'Eligible15.shp',
 'Ineligible15.shp',
 'XNB_housegroups.shp',
 'XNB_housegroups_Buffer.shp',
 'XNB_Outline.shp']

This section of the code is getting the unchange files in the loop and function, such as buffer and total study area

In [9]:
#First set slope to 15 and buffer the housegroup for the needed infield area

In [10]:
slope = 15
resiUnit = 'XNB_housegroups.shp'
outBuff = 'XNB_housegroups_Buffer'
arcpy.analysis.Buffer(resiUnit, outBuff, "20.8 Meters", "OUTSIDE_ONLY")

<Result 'C:\\Users\\1017d\\Box\\GIS_class_folders\\Final\\XNB_housegroups_Buffer.shp'>

In [17]:
#Use search cursor to retrive the total infield land area that can be entered in the excel
summed_total = 0
with arcpy.da.SearchCursor(outBuff, "Shape_Area") as cursor:
    for row in cursor:
        summed_total = summed_total + row[0]

#conver the result to squared kilometer       
sqkiloSummed_infield = summed_total * 0.000001
sqkiloSummed_infield

0.22789899940819996

In [11]:
#Use the slope tool in the spatial analyst licence to create the slope raster, this will be constant in the loop
inRaster = 'DEM.tif'
outMeasure = "PERCENT_RISE"
outSlope = Slope(inRaster, "PERCENT_RISE", 1, "PLANAR")

In [18]:
#set the study area outline for the use of creating eligible land
studyArea = 'XNB_Outline.shp'

#use search cursor to retrive the total study area and convert it to squared kilometer
with arcpy.da.SearchCursor(studyArea, ['SHAPE@AREA']) as cursor:
    for row in cursor:
        print(f'Area equal: {row[0] * 0.000001}')

Area equal: 2.194922027604409


The first set of code will be using a while loop start with testing 15% slope of the landscape, and increase 5% with each loop until it reaches 30%

In [19]:
while slope <= 30:
    #use the Con tool in the spatial analyst licence to generate polygon containing 15% slope
    whereClause = "VALUE < " + str(slope)
    outCon = Con(outSlope, 1, 0, whereClause)
    outPolygon = str(slope) + "polygon.shp"
    arcpy.conversion.RasterToPolygon(outCon, outPolygon, "NO_SIMPLIFY", "VALUE")
    print(f'Generating {slope} percent slope polygon')
    
    #select the polygon for the ineligible land based on the gridcode
    outLand = str(slope) + "_land.shp"
    selectClause = '"gridcode" = 1'
    arcpy.analysis.Select(outPolygon, outLand, selectClause)
    print(f'Creating {slope} percent ineligible land')
    
    #use union to combine the ineligible polygon with the housegroup and infield buffer
    inUnion = [outBuff, resiUnit, outLand]
    outUnion = "Ineligible" + str(slope) + ".shp"
    arcpy.analysis.Union(inUnion, outUnion)
    print(f'Union {slope} percent ineligible land')    
    
    #Erase the ineligible land from the maya site outline polygon to create eligible land
    outEligible = "Eligible" + str(slope) + ".shp"
    arcpy.analysis.Erase(studyArea, outUnion, outEligible)
    print(f'Creating {slope} percent eligible land')
    
    #use the search cursor to retrive the area of eligible land
    with arcpy.da.SearchCursor(outEligible, ['SHAPE@AREA']) as cursor:
        for row in cursor:
            print(f'Area equal: {row[0] * 0.000001}')
            
    slope += 5

Generating 15 percent slope polygon
Creating 15 percent ineligible land
Union 15 percent ineligible land
Creating 15 percent eligible land
Area equal: 0.9507070034350346
Generating 20 percent slope polygon
Creating 20 percent ineligible land
Union 20 percent ineligible land
Creating 20 percent eligible land
Area equal: 0.686160724994719
Generating 25 percent slope polygon
Creating 25 percent ineligible land
Union 25 percent ineligible land
Creating 25 percent eligible land
Area equal: 0.4807237306949918
Generating 30 percent slope polygon
Creating 30 percent ineligible land
Union 30 percent ineligible land
Creating 30 percent eligible land
Area equal: 0.3320785104123803


The second set of code is to create a function but not using any loop. This is made for analysising a specific number of slope e.g. 7%

In [20]:
def milpaSlope(n):
    '''
    This function is used to visualize the land "eligiblity" for milpa-cycle cultivation at Maya cities
    require one integer indicate what percentage of the land that is needed to be calculated
    
    Usage
    >>>milpaSlope(7)
    '''
    #use the Con tool in the spatial analyst licence to generate polygon containing 15% slope
    whereClause = "VALUE < " + str(n)
    outCon = Con(outSlope, 1, 0, whereClause)
    outPolygon = str(n) + "polygon.shp"
    arcpy.conversion.RasterToPolygon(outCon, outPolygon, "NO_SIMPLIFY", "VALUE")
    print(f'Generating {n} percent slope polygon')
    
    #select the polygon for the ineligible land based on the gridcode
    outLand = str(n) + "_land.shp"
    selectClause = '"gridcode" = 1'
    arcpy.analysis.Select(outPolygon, outLand, selectClause)
    print(f'Creating {n} percent ineligible land')
    
    #use union to combine the ineligible polygon with the housegroup and infield buffer
    inUnion = [outBuff, resiUnit, outLand]
    outUnion = "Ineligible" + str(n) + ".shp"
    arcpy.analysis.Union(inUnion, outUnion)
    print(f'Union {n} percent ineligible land')    
    
    #Erase the ineligible land from the maya site outline polygon to create eligible land
    outEligible = "Eligible" + str(n) + ".shp"
    arcpy.analysis.Erase(studyArea, outUnion, outEligible)
    print(f'Creating {n} percent eligible land')
    
    #use the search cursor to retrive the area of eligible land
    with arcpy.da.SearchCursor(outEligible, ['SHAPE@AREA']) as cursor:
        for row in cursor:
            print(f'Area equal: {row[0] * 0.000001}')

In [21]:
milpaSlope(7)

Generating 7 percent slope polygon
Creating 7 percent ineligible land
Union 7 percent ineligible land
Creating 7 percent eligible land
Area equal: 1.405920374216194
